In [1]:
import os
import sys
import time
import yaml
import numpy as np
import torch
import mmengine
from torch.utils.data import DataLoader
os.chdir('../')
from new_model.model import TrajectoryModel
from new_tools.dataset import TrajectoryDataset

In [ ]:
dataset_path = './data/CODA/'
dataset_name = 'coda'
config = yaml.load(open('./configs/coda.yaml', 'r'), Loader=yaml.FullLoader)
train_dataset = TrajectoryDataset(dataset_path, dataset_name, 'train')
val_dataset = TrajectoryDataset(dataset_path, dataset_name, 'val')
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=val_dataset.collate_fn)
model = TrajectoryModel(num_class=config['num_class'], in_size=2, obs_len=config['obs_len'], pred_len=config['pred_len'], 
                        embed_size=config['embed_size'], num_decode_layers=config['num_decode_layers'], num_modes=config['num_modes'])
model.cuda()

In [3]:
model.eval()
with torch.no_grad():
    for data_batch in val_dataloader:
        data_batch = [tensor.cuda() for tensor in data_batch]
        obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch
        preds, scores = model(obs, neis, nei_masks, self_labels, nei_labels)
        break

In [6]:
model.train()
for data_batch in train_dataloader:
    data_batch = [tensor.cuda() for tensor in data_batch]
    obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch
    preds, scores = model(obs, neis, nei_masks, self_labels, nei_labels)
    break

In [9]:
obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch

In [ ]:
obs.shape, futures.shape, neis.shape, nei_masks.shape, self_labels.shape, nei_labels.shape, refs.shape, rot_mats.shape

In [45]:
a = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]], dtype=torch.float32)
b = torch.tensor([1, 0])
# 取a的0批次的第1行和1批次的第0行
c = a[torch.arange(a.size(0)), b]

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [3], [2]

In [44]:
torch.arange(a.size(0)), b

(tensor([0, 1]), tensor([1, 0]))

In [41]:
c

tensor([[3., 4.],
        [5., 6.]])